In [1]:
import re, json
import random
import spacy 
import numpy as np
import en_core_web_sm
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
# Load the spacy model: nlp
nlp = en_core_web_sm.load()
lemmatizer = WordNetLemmatizer()
# !python -m spacy download en

## Preprocess query

In [51]:
def clean(text, pos=False):
    text = text.lower()
    text_list = [word for word in text.split()]
    #If last word has a question mark attached, that gets removed also.
    text_regex = [word for word in text_list if re.search('^[a-z0-9]+$',word)]
    final_text = [lemmatizer.lemmatize(word) for word in text_regex]
    if(pos):         #If POS is true, then attach tag
        final_text = nltk.pos_tag(final_text)
    return final_text

In [52]:
def listToString(inputList):
    return ' '.join(inputList)

## Intent-Entity match 

In [80]:
default_response = "Sorry, I could not understand you."

In [81]:
def get_response_from_intent(matched_intent):
    response = default_response
    for intent in intents['intents']:
        
        if(intent['tag']==matched_intent):
            response = random.choice(intent['responses'])
    return response

In [85]:
# import our chat-bot intents file
with open('data/intents.json') as json_data:
    intents = json.load(json_data)

In [86]:
patterns = {}
for intent in intents['intents']:
    patterns[intent['tag']] = re.compile('|'.join(intent['patterns']))
    
patterns

{'goodbye': re.compile(r'bye|see you later|goodbye', re.UNICODE),
 'greeting': re.compile(r'hi|how are you|is anyone there|hello|good day|hey',
 re.UNICODE),
 'thanks': re.compile(r"thanks|thank you|that's helpful", re.UNICODE)}

In [87]:
# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message) :
            matched_intent = intent
        
    return matched_intent

## Create a dictionary of rules and get responses

In [88]:
def get_num_groups(regex):
    return re.compile(regex).groups

In [89]:
rules = {}
for intent in intents['rules']:
    print(intent)
    rules[intent['patterns']] = intent['responses']
    
rules

{'tag': 'topN', 'patterns': 'what (is|are) the top (.*) hotel (.*)', 'responses': ['The Top {1} hotels {0} are: ']}
{'tag': 'review', 'patterns': 'what (is|are) the (review|reviews) for (.*)', 'responses': ['The review for {0}: ']}
{'tag': 'amenities', 'patterns': 'doe (.*) have good (.*)', 'responses': ['The review for {1}: ']}
{'tag': 'best', 'patterns': 'which is the best hotel (.*)', 'responses': ['The review for {0}: ']}


{'doe (.*) have good (.*)': ['The review for {1}: '],
 'what (is|are) the (review|reviews) for (.*)': ['The review for {0}: '],
 'what (is|are) the top (.*) hotel (.*)': ['The Top {1} hotels {0} are: '],
 'which is the best hotel (.*)': ['The review for {0}: ']}

In [90]:
# Define match_rule()
def match_rule(rules, message):
    response, phrase = default_response, None
    
    # Iterate over the rules dictionary
    for pattern, responseList in rules.items():
        # Create a match object
        match = re.search(pattern, message)
        num_groups = get_num_groups(pattern)
        
        if match is not None:
            # Choose a random response
            response = random.choice(responseList)
        
            if '{0}' in response:
                phrase = match.group(num_groups)
                response = response.replace('{0}', phrase)
                print(match.group(num_groups))
            
            if '{1}' in response:
                phrase = match.group(num_groups-1)
                response = response.replace('{1}', phrase)
                print(match.group(num_groups-1))
                
    # Return the response and phrase
    return response, phrase

# Test match_rule
# print(match_rule(rules, "What is the review for hotel Abra"))
# print(match_rule(rules, listToString(clean("What are the top 5 hotel in LA"))))
print(match_rule(rules, listToString(clean("Does hotel Abru Chu have good reviews"))))

print(listToString(clean("Does hotel Abru Chu have good reviews")))


hotel abru chu
('The review for hotel abru chu: ', 'hotel abru chu')
doe hotel abru chu have good review


In [91]:
def get_response(message):
    response="Sorry, I could not understand you"
    clean_message = clean(message)
    clean_string = listToString(clean_message)
    intent = match_intent(clean_string)
    if(intent is not None):
        response = get_response_from_intent(intent)
    else:
        response = match_rule(rules, clean_string)[0]
    return response

## Test the Chatbot

In [93]:
message = input()
response = get_response(message)
print(response)

Does Abru Chu have good reviews
abru chu
The review for abru chu: 
